## Token Merging for Stable Diffusion running with OpenVINO demo
This notebook demonstrates how to use Token Merging method to accelerate Stable Diffusion model running with OpenVINO. The method is applied to PyTorch model before exporting to OpenVINO representation.

In [ ]:
import tomeov
from diffusers import StableDiffusionPipeline, DDPMScheduler
from diffusers.training_utils import set_seed
from optimum.intel.openvino import OVStableDiffusionPipeline
from IPython.display import display

In [ ]:
scheduler = DDPMScheduler(beta_start=0.00085, beta_end=0.012,
                          beta_schedule="scaled_linear", num_train_timesteps=1000)
pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", scheduler=scheduler)
pipe.safety_checker = lambda images, clip_input: (images, False)


* Create a pipiline with Token Merging applied to a Stable Diffusion model and export it to OpenVINO representation.

In [ ]:
# Apply ToMe with a 30% merging ratio
tomeov.patch_stable_diffusion(pipe, ratio=0.3) # Can also use pipe.unet in place of pipe here

In [ ]:
save_dir = "stable_diffusion_optimized"
tomeov.export_diffusion_pipeline(pipe, save_dir)

* Create OpenVINO-based pipeline. We fix image size for faster inference.

In [ ]:
set_seed(42)
ov_pipe = OVStableDiffusionPipeline.from_pretrained(save_dir, compile=False)
ov_pipe.reshape(batch_size=1, height=512, width=512, num_images_per_prompt=1)
ov_pipe.compile()

* Generate and display the image.

In [ ]:
set_seed(42)
output = ov_pipe(prompt, num_inference_steps=50, output_type="pil")
display(output.images[0])